In [30]:
import requests

url = "https://realtor.p.rapidapi.com/properties/list-for-rent"

querystring = {"state_code":"NY","city":"New York City","limit":"200","offset":"0","sort":"relevance"}

headers = {
	"X-RapidAPI-Key": "7c62f71be8msh61ccb7db7073854p1a9640jsnd13a2fbfc675",
	"X-RapidAPI-Host": "realtor.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)


In [31]:
columns=['prop_type',
         'address_new',
         'price_raw',
         'price',
         'beds',
         'baths',
         'sqft',
         'rank',
         'source']

In [32]:
import numpy as np
import pandas as pd
data = response.json()
df = pd.DataFrame.from_dict(data['listings'])
df = df[columns]

In [33]:
pd.set_option('display.max_columns',50)

In [34]:
df.head()

,prop_type,address,address_new,prop_status,price_raw,is_showcase,price,beds,baths,sqft,short_price,rank,source
0,condop,"2 Test St Unit Vgt in West Brighton, Staten Is...","{'city': 'Staten Island', 'country': 'USA', 'c...",for_rent,1,False,$1/mo,1,2,sq ft N/A,$1/mo,1,mls
1,condop,"2 Test 1006R St Unit Auto in West Brighton, St...","{'city': 'Staten Island', 'country': 'USA', 'c...",for_rent,1,False,$1/mo,1,2,sq ft N/A,$1/mo,2,mls
2,condop,"2 Test 0906R St Unit Auto in West Brighton, St...","{'city': 'Staten Island', 'country': 'USA', 'c...",for_rent,1,False,$1/mo,1,2,sq ft N/A,$1/mo,3,mls
3,apartment,"50 W 34th St in Koreatown, New York, 10001","{'city': 'New York', 'country': 'USA', 'county...",for_rent,1,False,$1+/mo,S-1,1-2,439+ sq ft,$1+/mo,4,community
4,condop,"2 Test 1106R St Unit Auto in West Brighton, St...","{'city': 'Staten Island', 'country': 'USA', 'c...",for_rent,1,False,$1/mo,1,2,sq ft N/A,$1/mo,5,mls


In [35]:
df['prop_type'].value_counts()

condo             130
single_family      30
apartment          27
condop              5
townhome            3
multi_family        3
other               1
duplex_triplex      1
Name: prop_type, dtype: int64

In [36]:
import re
df['price'] = df['price'].apply(lambda x:re.sub("[^0-9]","",x))

#### Price = 1 is definitely a typo or a ridiculous mistake

In [37]:
df = df[df['price']!='1']

In [38]:
df['sqft'].value_counts()

sq ft N/A      121
650 sq ft        5
N/A sq ft        4
113 sq ft        3
114 sq ft        3
102 sq ft        2
106 sq ft        2
120 sq ft        2
145 sq ft        2
140 sq ft        2
500 sq ft        2
800 sq ft        2
135 sq ft        1
130 sq ft        1
121 sq ft        1
235 sq ft        1
780 sq ft        1
438+ sq ft       1
414+ sq ft       1
1,936 sq ft      1
139 sq ft        1
123 sq ft        1
190 sq ft        1
110 sq ft        1
138 sq ft        1
208 sq ft        1
527 sq ft        1
115 sq ft        1
178 sq ft        1
820 sq ft        1
166 sq ft        1
175 sq ft        1
136 sq ft        1
997 sq ft        1
1,521 sq ft      1
179 sq ft        1
600 sq ft        1
200 sq ft        1
155 sq ft        1
550+ sq ft       1
126 sq ft        1
165 sq ft        1
108 sq ft        1
128 sq ft        1
101 sq ft        1
315 sq ft        1
122 sq ft        1
2,436 sq ft      1
103 sq ft        1
107 sq ft        1
104 sq ft        1
111 sq ft        1
112 sq ft   

In [39]:
def na(x):
    if "N/A" in x: 
        return np.nan
    else:
        return re.sub('[^0-9]','',x)

In [40]:
df['sqft'] = df['sqft'].apply(na)

In [41]:
df['sqft'] = df['sqft'].astype(float)

In [42]:
df['sqft'].median()

140.0

In [43]:
df['sqft'] = df['sqft'].fillna(df['sqft'].median())

In [44]:
df['sqft']

8      121.0
9      140.0
10     140.0
11     140.0
12     179.0
       ...  
195    140.0
196    140.0
197    140.0
198    140.0
199    140.0
Name: sqft, Length: 192, dtype: float64

In [45]:
df['sqft'].value_counts()

140.0     127
650.0       5
113.0       3
114.0       3
120.0       2
500.0       2
106.0       2
800.0       2
102.0       2
145.0       2
123.0       1
235.0       1
130.0       1
438.0       1
414.0       1
135.0       1
1936.0      1
780.0       1
121.0       1
190.0       1
110.0       1
175.0       1
138.0       1
208.0       1
527.0       1
115.0       1
178.0       1
820.0       1
166.0       1
139.0       1
136.0       1
997.0       1
1521.0      1
179.0       1
600.0       1
200.0       1
155.0       1
550.0       1
126.0       1
165.0       1
108.0       1
128.0       1
101.0       1
315.0       1
122.0       1
2436.0      1
103.0       1
107.0       1
104.0       1
111.0       1
112.0       1
119.0       1
Name: sqft, dtype: int64

In [61]:
df.reset_index(inplace=True,drop=True)

In [58]:
df['address_new'][8]

{'city': 'Staten Island',
 'country': 'USA',
 'county': 'Richmond',
 'county_needed_for_uniq': False,
 'lat': 40.631516,
 'line': '5 Test St Unit 12TEST',
 'long': -74.12068,
 'postal_code': '10310',
 'state_code': 'NY',
 'state': 'New York',
 'time_zone': 'America/New_York'}

In [67]:
df['City']=None
df['Country']=None
df['State']=None
for i in range(0,len(df)):
    temp = df['address_new'][i]
    df['City'][i] = temp['city']
    df['Country'][i] = temp['country']
    df['State'][i] = temp['state']

C:\Users\Shubh\AppData\Local\Temp\ipykernel_4576\2022248598.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['City'][i] = temp['city']
C:\Users\Shubh\AppData\Local\Temp\ipykernel_4576\2022248598.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Country'][i] = temp['country']
C:\Users\Shubh\AppData\Local\Temp\ipykernel_4576\2022248598.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['State'][i] = temp['state']


In [68]:
df.head()

,prop_type,address,address_new,prop_status,price_raw,is_showcase,price,beds,baths,sqft,short_price,rank,source,City,Country,State
0,townhome,"5 Test St Unit 12TEST in West Brighton, Staten...","{'city': 'Staten Island', 'country': 'USA', 'c...",for_rent,101,False,101,2,3,121.0,$101/mo,9,mls,Staten Island,USA,New York
1,single_family,"Beach 3rd St in Far Rockaway, Far Rockaway, 11691","{'city': 'Far Rockaway', 'country': 'USA', 'co...",for_rent,150,False,150,S,Call,140.0,$150/mo,10,mls,Far Rockaway,USA,New York
2,single_family,"163 Cleveland St in Highland Park, Brooklyn, 1...","{'city': 'Brooklyn', 'country': 'USA', 'county...",for_rent,800,False,800,1,1,140.0,$800/mo,11,unit_rental,Brooklyn,USA,New York
3,single_family,"119-14 Rockaway Blvd in Richmond Hill Park, So...","{'city': 'South Ozone Park', 'country': 'USA',...",for_rent,800,False,800,S,1,140.0,$800/mo,12,mls,South Ozone Park,USA,New York
4,townhome,"267 Covert St Unit 14 in Bushwick, New York, 1...","{'city': 'New York', 'country': 'USA', 'county...",for_rent,850,False,850,1,1,179.0,$850/mo,13,unit_rental,New York,USA,New York


In [74]:
df.drop(columns=['address_new'],inplace=True)

In [84]:
df.beds.value_counts()

4      79
1      33
3      26
5      23
S      15
2       7
6       6
S-2     2
S-3     1
Name: beds, dtype: int64

In [88]:
dele = df[(df['baths']=='0-2') | (df['baths']=='1-2') | (df['baths']=='Call') | (df['beds']=='S') | (df['beds']=='S-2') | (df['beds']=='S-3')].index

In [89]:
len(dele)

18

In [91]:
df.drop(dele,inplace=True)

In [93]:
df

,prop_type,prop_status,price_raw,price,beds,baths,sqft,rank,source,City,Country,State
0,townhome,for_rent,101,101,2,3,121.0,9,mls,Staten Island,USA,New York
2,single_family,for_rent,800,800,1,1,140.0,11,unit_rental,Brooklyn,USA,New York
4,townhome,for_rent,850,850,1,1,179.0,13,unit_rental,New York,USA,New York
5,single_family,for_rent,900,900,1,1,140.0,14,unit_rental,Brooklyn,USA,New York
6,apartment,for_rent,945,945,1,1,140.0,15,unit_rental,Brooklyn,USA,New York
...,...,...,...,...,...,...,...,...,...,...,...,...
187,condo,for_rent,1475,1475,4,2,140.0,24,unit_rental,New York,USA,New York
188,condo,for_rent,1475,1475,4,2,140.0,25,unit_rental,New York,USA,New York
189,condo,for_rent,1475,1475,3,1,140.0,26,unit_rental,New York,USA,New York
190,condo,for_rent,1475,1475,4,2,140.0,27,unit_rental,New York,USA,New York


In [94]:
df.to_csv('Rental_Cleaned.csv')